## Baseline-net

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 26397 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## Loading the data

In [3]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.datasets import cifar100

Using TensorFlow backend.


In [4]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='fine')
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.1, 
                                                  random_state=1974,
                                                  stratify = y_train)

169009152/169001437 [==============================] - 53s 0us/step


In [0]:
train_features = X_train.reshape(X_train.shape[0], 32, 32, 3)/255
vali_features = X_val.reshape(X_val.shape[0], 32, 32, 3)/255
test_features = X_test.reshape(X_test.shape[0], 32, 32, 3)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Training the model

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [0]:
class TimingCallback(keras.callbacks.Callback):
    
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [0]:
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)

num_classes_fine = 100

img_input = Input(shape=input_shape, name='input')


x = Conv2D(64, (3, 3), activation='relu', name='block1_conv1', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', name='block1_conv2', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), name='block1_pool')(x)

x = Conv2D(128, (3, 3), activation='relu', name='block1_conv3', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', name='block1_conv4', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

x = Conv2D(256, (3, 3), activation='relu', name='block1_conv5', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', name='block1_conv6', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), name='block3_pool')(x)

x = Conv2D(512, (3, 3), activation='relu', name='block1_conv7', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', name='block1_conv8', padding='same')(x)
x = BatchNormalization()(x)

x = Flatten(name='flatten')(x)
x = Dense(1024, name='fc_1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(1024, name='fc_2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)


fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(x)

In [35]:
model = Model(inputs=img_input, outputs= fine_pred, name='baseline_net')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block1_conv3 (Conv2D)        (None, 16, 16, 128)       73856     
__________

In [0]:
# adam = optimizers.Adam(lr=0.00025)
sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
clr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
csv_cb = CSVLogger('drive/TCC-ITAU/cifar-100/training-data/baseline/baseline-net-simple.csv', separator=',', append=False)


cbks = [clr_cb, time_cb, csv_cb]

In [38]:
epochs = 50
start_time = time()
training = model.fit(train_features, y_train_cat,
                     validation_data=(vali_features,  y_val_cat),
                     epochs=epochs,
                     batch_size=256, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 36s 803us/step - loss: 4.7780 - acc: 0.0424 - val_loss: 3.9832 - val_acc: 0.1190
Epoch 2/50
45000/45000 [==============================] - 33s 730us/step - loss: 4.1616 - acc: 0.0952 - val_loss: 3.6922 - val_acc: 0.1586
Epoch 3/50
45000/45000 [==============================] - 33s 729us/step - loss: 3.9004 - acc: 0.1266 - val_loss: 3.4595 - val_acc: 0.1920
Epoch 4/50
45000/45000 [==============================] - 33s 728us/step - loss: 3.6867 - acc: 0.1569 - val_loss: 3.3335 - val_acc: 0.2196
Epoch 5/50
45000/45000 [==============================] - 33s 728us/step - loss: 3.5123 - acc: 0.1845 - val_loss: 3.2063 - val_acc: 0.2400
Epoch 6/50
45000/45000 [==============================] - 33s 728us/step - loss: 3.3541 - acc: 0.2074 - val_loss: 3.0683 - val_acc: 0.2644
Epoch 7/50
45000/45000 [==============================] - 33s 730us/step - loss: 3.2133 - acc: 0.2324 - val_loss: 3.0

In [39]:
model.evaluate(test_features, y_test_cat)

10000/10000 [==============================] - 5s 484us/step


[2.1461879428863524, 0.4726]

In [40]:
print('Total training time: {}'.format(training_time))

Total training time: 1682.5896458625793


In [0]:
model.save('drive/TCC-ITAU/cifar-100/models/baseline-net.h5')